In [1]:
# Cell 1: Setup (paths + libraries) — repo-local, reproducible

source("../setup/r_bootstrap.R")

Repo root: /Users/tommyrucinski/dev/repos/tcga-brca-luminalA-deg-gsea 
The following package(s) are in an inconsistent state:

 package     installed recorded used
 bit         y         y        ?   
 bit64       y         y        ?   
 clipr       y         y        ?   
 fgsea       n         n        y   
 ggrepel     n         n        y   
 hms         y         y        ?   
 lattice     y         n        y   
 Matrix      y         n        y   
 msigdbr     n         n        y   
 prettyunits y         y        ?   
 progress    y         y        ?   
 readr       y         y        ?   
 survival    y         n        y   
 survminer   n         n        y   
 tzdb        y         y        ?   
 vroom       y         y        ?   

See `?renv::status` for advice on resolving these issues.
renv is out-of-sync. Restoring from renv.lock...
- The library is already synchronized with the lockfile.
✅ R bootstrap complete.


In [2]:
# Cell 2: Define Paths and Load Data

options(warnPartialMatchDollar = TRUE)

meta_fp <- file.path(REPO_ROOT, "data/processed/preprocessing_outputs/metadata_LumA_IDC_Tumor_vs_AllNormals.tsv")
expr_fp <- file.path(REPO_ROOT, "data/processed/preprocessing_outputs/expr_LumA_IDC_Tumor_vs_AllNormals.tsv")

cat("meta_fp:", meta_fp, "\n")
cat("expr_fp:", expr_fp, "\n")
cat("meta exists:", file.exists(meta_fp), "\n")
cat("expr exists:", file.exists(expr_fp), "\n")

meta <- read.delim(meta_fp, stringsAsFactors = FALSE, check.names = FALSE)
expr <- read.delim(expr_fp, row.names = 1, check.names = FALSE)

print(colnames(meta))
cat("Metadata:", nrow(meta), "×", ncol(meta), "\n")
cat("Expression:", nrow(expr), "×", ncol(expr), "\n")
print(table(meta$Group))

stopifnot("Sample" %in% colnames(meta))
stopifnot(all(colnames(expr) %in% meta$Sample))

meta <- meta[match(colnames(expr), meta$Sample), ]
stopifnot(all(meta$Sample == colnames(expr)))

meta_fp: /Users/tommyrucinski/dev/repos/tcga-brca-luminalA-deg-gsea/data/processed/preprocessing_outputs/metadata_LumA_IDC_Tumor_vs_AllNormals.tsv 
expr_fp: /Users/tommyrucinski/dev/repos/tcga-brca-luminalA-deg-gsea/data/processed/preprocessing_outputs/expr_LumA_IDC_Tumor_vs_AllNormals.tsv 
meta exists: TRUE 
expr exists: TRUE 
 [1] "Sample"             "Group"              "molecular_subtype" 
 [4] "sample_type"        "histological_type"  "OS_time"           
 [7] "OS_event"           "ER_status"          "PR_status"         
[10] "HER2_status"        "stage"              "T_stage"           
[13] "N_stage"            "age_at_diagnosis"   "integrated_cluster"
[16] "tcga_sample_code"  
Metadata: 413 × 16 
Expression: 20530 × 413 

Normal  Tumor 
   114    299 


In [3]:
# Cell 3: Data Pre-processing for limma (strict alignment)

# 1) Verify exact overlap
missing_in_expr <- setdiff(meta$Sample, colnames(expr))
missing_in_meta <- setdiff(colnames(expr), meta$Sample)

if (length(missing_in_expr) > 0) {
  cat("Samples in meta but missing in expr:", length(missing_in_expr), "\n")
  print(head(missing_in_expr, 10))
  stop("Metadata contains samples not present in expression matrix.")
}

if (length(missing_in_meta) > 0) {
  cat("Samples in expr but missing in meta:", length(missing_in_meta), "\n")
  print(head(missing_in_meta, 10))
  stop("Expression matrix contains samples not present in metadata.")
}

# 2) Reorder expression columns to match metadata order exactly
expr <- expr[, meta$Sample, drop = FALSE]

# 3) Confirm exact alignment
stopifnot(all(colnames(expr) == meta$Sample))

# 4) Build design matrix (Normal = reference)
group <- factor(meta$Group, levels = c("Normal", "Tumor"))
design <- model.matrix(~ group)
colnames(design) <- c("Intercept", "Tumor_vs_Normal")

cat("Design matrix columns:\n")
print(colnames(design))
print(table(group))

Design matrix columns:
[1] "Intercept"       "Tumor_vs_Normal"
group
Normal  Tumor 
   114    299 


In [4]:
# Cell 4: limma-trend + save DEG results

suppressPackageStartupMessages({
  library(limma)
})

# Ensure matrix
expr_mat <- as.matrix(expr)

# Fit model
fit <- lmFit(expr_mat, design)
fit <- eBayes(fit, trend = TRUE)

# Extract all genes (Tumor vs Normal)
# Use the coefficient name "Tumor_vs_Normal"
tt <- topTable(fit, coef = "Tumor_vs_Normal", number = Inf, sort.by = "P")

# Standardized output table
res <- data.frame(
  gene = rownames(tt),
  log2FoldChange = tt$logFC,
  AveExpr = tt$AveExpr,
  t_stat = tt$t,
  pvalue = tt$P.Value,
  padj = tt$adj.P.Val,
  B_statistic = tt$B,
  stringsAsFactors = FALSE
)

# Output directory under results/
deg_dir <- file.path(REPO_ROOT, "results/tables/deg")
dir.create(deg_dir, recursive = TRUE, showWarnings = FALSE)

out_csv <- file.path(deg_dir, "DEG_Results_LumA_IDC_Tumor_vs_AllNormals_limma.csv")
write.csv(res, out_csv, row.names = FALSE)

cat("Wrote DEG results:", out_csv, "\n")
cat("Significant DEGs (FDR<0.05 & |log2FC|>1):",
    sum(res$padj < 0.05 & abs(res$log2FoldChange) > 1, na.rm = TRUE), "\n")

Warning message:
“Zero sample variances detected, have been offset away from zero”


Wrote DEG results: /Users/tommyrucinski/dev/repos/tcga-brca-luminalA-deg-gsea/results/tables/deg/DEG_Results_LumA_IDC_Tumor_vs_AllNormals_limma.csv 
Significant DEGs (FDR<0.05 & |log2FC|>1): 4191 


In [ ]:
# Cell 5: fgsea - Load gene sets from MSigDB

suppressPackageStartupMessages({
  library(fgsea)
  library(msigdbr)
  library(dplyr)
})

# Get gene sets from MSigDB (using updated API - collection/subcollection)
cat("Loading gene sets from MSigDB...\n")

# Hallmark gene sets (curated functional signatures)
hallmark_df <- msigdbr(species = "Homo sapiens", collection = "H")
pathways_hallmark <- split(hallmark_df$gene_symbol, hallmark_df$gs_name)
cat("  Hallmark:", length(pathways_hallmark), "gene sets\n")

# KEGG Legacy pathways
kegg_df <- msigdbr(species = "Homo sapiens", collection = "C2", subcollection = "CP:KEGG_LEGACY")
pathways_kegg <- split(kegg_df$gene_symbol, kegg_df$gs_name)
cat("  KEGG Legacy:", length(pathways_kegg), "gene sets\n")

# Reactome pathways
reactome_df <- msigdbr(species = "Homo sapiens", collection = "C2", subcollection = "CP:REACTOME")
pathways_reactome <- split(reactome_df$gene_symbol, reactome_df$gs_name)
cat("  Reactome:", length(pathways_reactome), "gene sets\n")

# GO Biological Process
go_bp_df <- msigdbr(species = "Homo sapiens", collection = "C5", subcollection = "GO:BP")
pathways_gobp <- split(go_bp_df$gene_symbol, go_bp_df$gs_name)
cat("  GO:BP:", length(pathways_gobp), "gene sets\n")

cat("\nTotal gene sets loaded:", 
    length(pathways_hallmark) + length(pathways_kegg) + 
    length(pathways_reactome) + length(pathways_gobp), "\n")

In [ ]:
# Cell 6: fgsea - Run enrichment analysis

# Create ranked gene list from t-statistics (same as Broad GSEA uses)
ranks <- setNames(res$t_stat, res$gene)
ranks <- ranks[!is.na(ranks)]
ranks <- sort(ranks, decreasing = TRUE)

cat("Ranked list:", length(ranks), "genes\n")
cat("Range: [", min(ranks), ",", max(ranks), "]\n\n")

# Run fgsea for each gene set collection
# eps=0 gives exact p-values (slower but more accurate for small sets)
set.seed(42)

cat("Running fgsea on Hallmark...\n")
fgsea_hallmark <- fgsea(pathways = pathways_hallmark, stats = ranks, eps = 0, minSize = 15, maxSize = 500)
fgsea_hallmark$Database <- "Hallmark"

cat("Running fgsea on KEGG...\n")
fgsea_kegg <- fgsea(pathways = pathways_kegg, stats = ranks, eps = 0, minSize = 15, maxSize = 500)
fgsea_kegg$Database <- "KEGG"

cat("Running fgsea on Reactome...\n")
fgsea_reactome <- fgsea(pathways = pathways_reactome, stats = ranks, eps = 0, minSize = 15, maxSize = 500)
fgsea_reactome$Database <- "Reactome"

cat("Running fgsea on GO:BP...\n")
fgsea_gobp <- fgsea(pathways = pathways_gobp, stats = ranks, eps = 0, minSize = 15, maxSize = 500)
fgsea_gobp$Database <- "GO_BP"

cat("\nfgsea complete.\n")

In [ ]:
# Cell 7: fgsea - Combine and save results

# Combine all results
fgsea_all <- bind_rows(fgsea_hallmark, fgsea_kegg, fgsea_reactome, fgsea_gobp)

# Add direction annotation
fgsea_all <- fgsea_all %>%
  mutate(
    direction = case_when(
      NES > 0 ~ "Up_in_Tumor",
      NES < 0 ~ "Down_in_Tumor",
      TRUE ~ "Neutral"
    )
  ) %>%
  arrange(desc(NES))

# Summary statistics
cat("fgsea Results Summary:\n")
cat("=" %>% rep(50) %>% paste(collapse = ""), "\n")

for (db in c("Hallmark", "KEGG", "Reactome", "GO_BP")) {
  sub <- fgsea_all %>% filter(Database == db)
  n_sig <- sum(sub$padj < 0.05, na.rm = TRUE)
  n_up <- sum(sub$padj < 0.05 & sub$NES > 0, na.rm = TRUE)
  n_down <- sum(sub$padj < 0.05 & sub$NES < 0, na.rm = TRUE)
  cat(sprintf("  %s: %d significant (FDR<0.05) - %d up, %d down\n", db, n_sig, n_up, n_down))
}

# Save combined results
gsea_dir <- file.path(REPO_ROOT, "results/tables/gsea")
dir.create(gsea_dir, recursive = TRUE, showWarnings = FALSE)

# Convert leadingEdge list to string for CSV compatibility
fgsea_export <- fgsea_all %>%
  mutate(leadingEdge = sapply(leadingEdge, paste, collapse = ";"))

fgsea_out_fp <- file.path(gsea_dir, "fgsea_results_all_databases.csv")
write.csv(fgsea_export, fgsea_out_fp, row.names = FALSE)
cat("\nSaved fgsea results:", fgsea_out_fp, "\n")

# Also save .rnk file for compatibility with external tools
rnk <- data.frame(gene = names(ranks), t_stat = ranks)
rnk_fp <- file.path(gsea_dir, "LumA_IDC_Tumor_vs_AllNormals.rnk")
write.table(rnk, rnk_fp, sep = "\t", col.names = FALSE, row.names = FALSE, quote = FALSE)
cat("Saved .rnk file:", rnk_fp, "\n")

In [ ]:
# Cell 8: fgsea - Display top enriched pathways

library(knitr)

# Top 10 up-regulated pathways (Hallmark)
cat("\n### Top 10 Up-regulated Hallmark Pathways ###\n\n")
top_up_hallmark <- fgsea_hallmark %>%
  filter(NES > 0) %>%
  arrange(padj) %>%
  head(10) %>%
  select(pathway, NES, padj, size) %>%
  mutate(
    pathway = gsub("HALLMARK_", "", pathway),
    NES = round(NES, 2),
    padj = formatC(padj, format = "e", digits = 2)
  )
print(kable(top_up_hallmark))

# Top 10 down-regulated pathways (Hallmark)
cat("\n### Top 10 Down-regulated Hallmark Pathways ###\n\n")
top_down_hallmark <- fgsea_hallmark %>%
  filter(NES < 0) %>%
  arrange(padj) %>%
  head(10) %>%
  select(pathway, NES, padj, size) %>%
  mutate(
    pathway = gsub("HALLMARK_", "", pathway),
    NES = round(NES, 2),
    padj = formatC(padj, format = "e", digits = 2)
  )
print(kable(top_down_hallmark))

# Top KEGG pathways
cat("\n### Top 10 KEGG Pathways (by |NES|) ###\n\n")
top_kegg <- fgsea_kegg %>%
  arrange(padj) %>%
  head(10) %>%
  select(pathway, NES, padj, size) %>%
  mutate(
    pathway = gsub("KEGG_", "", pathway),
    NES = round(NES, 2),
    padj = formatC(padj, format = "e", digits = 2)
  )
print(kable(top_kegg))

In [ ]:
# Cell 9: Session info and available objects for downstream analysis

cat("Objects available for visualization notebook (03):\n")
cat("  - res: DEG results data.frame with gene, log2FoldChange, t_stat, padj\n")
cat("  - ranks: named vector of t-statistics for fgsea\n")
cat("  - fgsea_all: combined fgsea results for all databases\n")
cat("  - fgsea_hallmark, fgsea_kegg, fgsea_reactome, fgsea_gobp: individual results\n")
cat("  - pathways_*: gene set lists for each database\n")
cat("\nFiles saved:\n")
cat("  - results/tables/deg/DEG_Results_LumA_IDC_Tumor_vs_AllNormals_limma.csv\n")
cat("  - results/tables/gsea/fgsea_results_all_databases.csv\n")
cat("  - results/tables/gsea/LumA_IDC_Tumor_vs_AllNormals.rnk\n")

cat("\n\n")
sessionInfo()